In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets evaluate accelerate bitsandbytes SentencePiece

In [ ]:
import bitsandbytes
import accelerate
import sentencepiece

from transformers import  LlamaForSequenceClassification

import warnings
warnings.filterwarnings("ignore")

#### Data Wrangling

In [4]:
from transformers import LlamaTokenizer
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [ ]:
# import pandas as pd
# data = pd.read_csv('data/toxic_ru.csv', names = ['text', 'label'])
# data.drop(index = 0, inplace = True)

# train_texts, train_labels = data.text.values, data.label.values

# from sklearn.model_selection import train_test_split
# train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

# train = pd.DataFrame()
# test = pd.DataFrame()

# val_labels = [int(float(s)) for s in val_labels]
# train_labels = [int(float(s)) for s in train_labels]

# train['text'] = train_texts
# train['labels'] = train_labels

# test['text'] = val_texts
# test['labels'] = val_labels

# train.to_csv('data/train.csv', index = False)
# test.to_csv('data/test.csv', index = False)

In [5]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files={'train': '/content/drive/MyDrive/ds/github/llama/data/toxic/train.csv', 
                                          'test': '/content/drive/MyDrive/ds/github/llama/data/toxic/test.csv'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-09a2006ec3579714/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
dataset_sample = dataset['train'].shuffle(seed=42).select(range(10))
dataset_sample[:3]

{'text': ['нормальные люди от юмора смеются, а ты тут разводишь неукротимую диарею! Может, тебе не здесь?\n',
  'А я вот поэтому пин-код записываю на карте, шифром, при этом ключ от шифра помню отлично. Так что если забыл всегда могу посмотреть\n',
  'Никто не передаёт тепло воздухом, его передают водой. Это компактно. Опустить температуру в системе отопления до 4 (если сильно жарко снаружи) - в чём проблема?\n'],
 'labels': [1, 0, 0]}

In [7]:
def lowercase_condition(example):
    return {"text": example["text"].lower()}

def compute_comment_length(example):
    return {"comment_length": len(example["text"].split())}

In [8]:
dataset = dataset.map(lowercase_condition)
dataset = dataset.map(compute_comment_length)

Map:   0%|          | 0/11529 [00:00<?, ? examples/s]

Map:   0%|          | 0/2883 [00:00<?, ? examples/s]

Map:   0%|          | 0/11529 [00:00<?, ? examples/s]

Map:   0%|          | 0/2883 [00:00<?, ? examples/s]

In [9]:
dataset["train"][:3]

{'text': ['меня не интересует причина, меня беспокоят последствия\n',
  'может кто пояснить, за что минус?? т.к. мне это реально интересно?\n',
  'упс. поинтересовался. там ещё станочек прикупить можно: стоимость установки для водородной воды составляет примерно 50 тыс. руб. также можно выбрать генератор водородной воды, который может применяться несколькими персонами и стоит около 5 тыс. руб. это карманные модели\n'],
 'labels': [0, 0, 0],
 'comment_length': [7, 11, 37]}

In [24]:
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#     model.resize_token_embeddings(len(tokenizer))
    
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [25]:
tokenized_data = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/11529 [00:00<?, ? examples/s]

Map:   0%|          | 0/2883 [00:00<?, ? examples/s]

In [26]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model.config.vocab_size, len(tokenizer)   

(32000, 32001)

### Fine-Tuning

In [ ]:
model = LlamaForSequenceClassification.from_pretrained(
    "decapoda-research/llama-7b-hf",
    low_cpu_mem_usage = True,
    num_labels = 2,
    load_in_8bit = True,
    device_map='auto'
)

In [22]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

Using pad_token, but it is not set yet.


In [23]:
model.config.vocab_size, len(tokenizer)   

(32001, 32001)

In [ ]:
#model.resize_token_embeddings(len(tokenizer))

Embedding(32001, 4096)

In [27]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [28]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [29]:
trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:1644 in train                     │
│                                                                                                  │
│   1641 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1642 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1643 │   │   )                                                                                 │
│ ❱ 1644 │   │   return inner_training_loop(                                                       │
│   1645 │   │   │   args=args,                                                                    │
│   1646 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1647 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:1911 in _inner_training_loop      │
│                                                                                                  │
│   1908 │   │   │   │   │   with model.no_sync():                                                 │
│   1909 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1910 │   │   │   │   else:                                                                     │
│ ❱ 1911 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1912 │   │   │   │                                                                             │
│   1913 │   │   │   │   if (                                                                      │
│   1914 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:2657 in training_step             │
│                                                                                                  │
│   2654 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2655 │   │                                                                                     │
│   2656 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2657 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2658 │   │                                                                                     │
│   2659 │   │   if self.args.n_gpu > 1:                                                           │
│   2660 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/trainer.py:2689 in compute_loss              │
│                                                                                                  │
│   2686 │   │   │   labels = inputs.pop("labels")                                                 │
│   2687 │   │   else:                                                                             │
│   2688 │   │   │   labels = None                                                                 │
│ ❱ 2689 │   │   outputs = model(**inputs)                                                         │
│   2690 │   │   # Save past state if it exists              